## Lesson 10 - Experiment Toolkit
**미션 개요**
- 이번 실습 자료에서는 강의시간에 다루었던 Tensorboard & wandb 연동 코드에 대해 실습을 진행합니다.

**미션의 목적 및 배경**
- 학습이 진행되는 와중에 현재 모델의 다양한 metric, loss 들이 어떤 양상으로 변화하고 그 시각화된 결과를 눈으로 확인하는 것은 매우 유용합니다.
- 모델 학습의 trial / error 를 많이 줄여주고 모델링의 디버깅 및 ideation 에도 정말 많은 도움이 됩니다.
- 이를 편하게 하기 위한 방법으로 Tensorboard, wandb 와 같은 툴들을 사용할 수 있습니다.


**미션 수행으로 얻어갈 수 있는 역량**
- 모델의 Training 코드에 다양한 로깅을 추가하고, 이를 Tensorboard, wandb 를 통해 모니터링 하는 방법에 대해 학습합니다.

**미션 핵심 내용**
- 모델의 현재 학습 양상에 대해 실시간 모니터링을 하기 위한 툴들을 학습합니다.

**데이터셋 개요 및 저작권 정보**
- Face with Masks
- 캠프 교육용 라이선스 : 교육 내에서만 활용해 주시고, 일부 이미지라도 외부에 노출되는 일은 없도록 부탁드립니다. (데이터셋에 대한 설명글이나, 데이터셋을 활용하여 학습한 weight 정도가 공개 가능합니다)


1. 실습을 진행하기 앞서 먼저 'pip install wandb'를 통해 wandb를 설치하시고, 'wandb.ai' 홈페이지에서 회원가입을 진행 합니다.
2. 터미널 창에서 wandb login 을 입력하여 사용자 계정과 로컬 환경을 연결합니다.
    - 사용자 계정은 wandb.ai 홈페이지에서 로그인 후 profile - Settings - API Keys 항목에서 API키를 생성하여 연결할 수 있습니다. 최초 한번 진행됩니다.

In [ ]:
# !pip install wandb

In [ ]:
import random
import os, sys

import numpy as np
import torch
from torch.utils.data import Subset
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.tensorboard import SummaryWriter
import torchvision

from sklearn.model_selection import StratifiedKFold

sys.path.append(os.path.abspath('..'))

# BaseLine 코드로 주어진 dataset.py model.py, loss.py를 Import 합니다.
from dataset import MaskBaseDataset, BaseAugmentation
from model import *
from loss import create_criterion

sys.path.append('../')

import wandb

def seed_everything(seed):
    """
    동일한 조건으로 학습을 할 때, 동일한 결과를 얻기 위해 seed를 고정시킵니다.
    
    Args:
        seed: seed 정수값
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
seed_everything(42)

### Model Parameter Setting

In [ ]:
# -- parameters
# img_root = '학습 이미지 폴더의 경로를 입력해주세요.'

val_split = 0.4  # validation dataset의 비율
batch_size = 64
num_workers = 4
num_classes = 18

num_epochs = 100  # 학습할 epoch의 수
lr = 1e-4
lr_decay_step = 10
criterion_name = 'cross_entropy'

train_log_interval = 20  # logging할 iteration의 주기
name = "02_model_results"  # 결과를 저장하는 폴더의 이름

# -- settings
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

### wandb init
- wandb를 사용하기 앞서 먼저 초기화를 진행하는데 이 때, 모델 학습에 사용할 파라미터를 같이 전달할 수 있습니다.

In [ ]:
# -- wandb initialize with configuration
wandb.init(config={"batch_size": batch_size,
                   "lr"        : lr,
                   "epochs"    : num_epochs,
                   "name"      : name,
                   "criterion_name" : criterion_name})

### Training process

In [ ]:
from torchvision.models import vgg19_bn

# 데이터셋 생성
data_dir = '학습 이미지 폴더의 경로를 입력해주세요.'
dataset = MaskBaseDataset(data_dir)

# Augmentation Transform 생성
transform = BaseAugmentation(
    resize=[128, 96],
    mean=dataset.mean,
    std=dataset.std,
)

# 데이터셋 준비
dataset.set_transform(transform)
n_val = int(len(dataset) * val_split)
n_train = len(dataset) - n_val
train_set, val_set = torch.utils.data.random_split(dataset, [n_train, n_val])

# Train Valid DataLoader 생성
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=True,
    shuffle=True
)

val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=True,
    shuffle=False
)

# -- model
model = vgg19_bn(pretrained=True)
model.classifier = nn.Sequential(
    nn.Linear(512 * 7 * 7, 4096),
    nn.ReLU(True),
    nn.Dropout(),
    nn.Linear(4096, 4096),
    nn.ReLU(True),
    nn.Dropout(),
    nn.Linear(4096, num_classes),
)
model.to(device)

# -- loss & metric
criterion = create_criterion(criterion_name)
train_params = [{'params': getattr(model, 'features').parameters(), 'lr': lr / 10, 'weight_decay':5e-4},
                {'params': getattr(model, 'classifier').parameters(), 'lr': lr, 'weight_decay':5e-4}]
optimizer = Adam(train_params)
scheduler = StepLR(optimizer, lr_decay_step, gamma=0.5)

### Tensorboard
- Tensorboard는 먼저 SummaryWriter 객체에 log_dir 인자(로그를 저장할 디렉토리 경로)를 전달하여 로그를 저장할 준비를 합니다. 
    - 특정 주기마다 logger.add_scaler(이름, 값, 글로벌 스텝)을 전달하여 스칼라 값 로그를 기록합니다. 
    - 특정 주기마다 logger.add_image(이미지 그리드)를 전달하여 이미지 로그를 기록합니다. 
        - 여기에서는 train step에서 각 이미지들이 어떤식으로 Transform 되는지 기록해보겠습니다.
- 터미널에서 tensorboard --logdir='로그를 저장한 디렉토리 경로' 를 입력하여 텐서보드를 실행해 로그 기록을 확인할 수 있습니다.

### wandb
- wandb는 이전에 init 함수를 통해 초기화를 마쳤으므로 특정 주기마다 wandb.log({이름: 값, ...})를 전달하여 로그를 기록합니다.
- wandb.ai 홈페이지에서 로그 기록을 확인할 수 있습니다.

In [ ]:
os.makedirs(os.path.join(os.getcwd(), 'results', name), exist_ok=True)

counter = 0
patience = 10
accumulation_steps = 2
best_val_acc = 0
best_val_loss = np.inf

# Tensorboard 로그를 저장할 경로 지정
logger = SummaryWriter(log_dir=f"logdir/{name}")
for epoch in range(num_epochs):
    # train loop
    model.train()
    loss_value = 0
    matches = 0
    
    for idx, train_batch in enumerate(train_loader):
        inputs, labels = train_batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        outs = model(inputs)
        preds = torch.argmax(outs, dim=-1)
        loss = criterion(outs, labels)

        loss.backward()
        
        # -- Gradient Accumulation
        if (idx+1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        loss_value += loss.item()
        matches += (preds == labels).sum().item()
        if (idx + 1) % train_log_interval == 0:
            train_loss = loss_value / train_log_interval
            train_acc = matches / batch_size / train_log_interval
            current_lr = scheduler.get_last_lr()
            print(
                f"Epoch[{epoch}/{num_epochs}]({idx + 1}/{len(train_loader)}) || "
                f"training loss {train_loss:4.4} || training accuracy {train_acc:4.2%} || lr {current_lr}"
            )
            
            # Tensorboard 학습 단계에서 Loss, Accuracy 로그 저장
            logger.add_scalar("Train/loss", train_loss, epoch * len(train_loader) + idx)
            logger.add_scalar("Train/accuracy", train_acc, epoch * len(train_loader) + idx)

            loss_value = 0
            matches = 0
            
            # wandb 학습 단계에서 Loss, Accuracy 로그 저장
            wandb.log({
                "Train loss": train_loss,
                "Train acc" : train_acc
            })
    
    # 각 에폭의 마지막 input 이미지로 grid view 생성
    img_grid = torchvision.utils.make_grid(inputs)
    # Tensorboard에 train input 이미지 기록
    logger.add_image(f'{epoch}_train_input_img', img_grid, epoch)

    scheduler.step()

    # val loop
    with torch.no_grad():
        print("Calculating validation results...")
        model.eval()
        val_loss_items = []
        val_acc_items = []
        for val_batch in val_loader:
            inputs, labels = val_batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)

            loss_item = criterion(outs, labels).item()
            acc_item = (labels == preds).sum().item()
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)

        val_loss = np.sum(val_loss_items) / len(val_loader)
        val_acc = np.sum(val_acc_items) / len(val_set)
        
        # Callback1: validation accuracy가 향상될수록 모델을 저장합니다.
        if val_loss < best_val_loss:
            best_val_loss = val_loss
        if val_acc > best_val_acc:
            print("New best model for val accuracy! saving the model..")
            torch.save(model.state_dict(), f"results/{name}/{epoch:03}_accuracy_{val_acc:4.2%}.ckpt")
            best_val_acc = val_acc
            counter = 0
        else:
            counter += 1
        # Callback2: patience 횟수 동안 성능 향상이 없을 경우 학습을 종료시킵니다.
        if counter > patience:
            print("Early Stopping...")
            break
        
        print(
            f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
            f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}"
        )
        # Tensorboard 검증 단계에서 Loss, Accuracy 로그 저장
        logger.add_scalar("Val/loss", val_loss, epoch)
        logger.add_scalar("Val/accuracy", val_acc, epoch)

        # wandb 검증 단계에서 Loss, Accuracy 로그 저장
        wandb.log({
            "Valid loss": val_loss,
            "Valid acc" : val_acc
        })
      

###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.